In [13]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from gurobipy import *

In [14]:
distances = pd.read_excel('data/distances.xlsx', index_col=0)
distances = distances.fillna(0)
distances = distances + distances.T
distances -= np.eye(distances.shape[0]) * 3


events18 = pd.read_csv('data/events18.csv',sep='\t',index_col=0)
events18["Start"] = pd.to_datetime(events18["Start"])
events18["End"] = pd.to_datetime(events18["End"])

events19 = pd.read_csv('data/events19.csv',sep='\t',index_col=0)
events19["Start"] = pd.to_datetime(events19["Start"])
events19["End"] = pd.to_datetime(events19["End"])

events20 = pd.read_csv('data/events20.csv',sep='\t',index_col=0)
events20["Start"] = pd.to_datetime(events20["Start"])
events20["End"] = pd.to_datetime(events20["End"])

events21 = pd.read_csv('data/events21.csv',sep='\t',index_col=0)
events21["Start"] = pd.to_datetime(events21["Start"])
events21["End"] = pd.to_datetime(events21["End"])

sallesDeCours = pd.read_csv('data/SallesDeCours.csv',sep=';')

students18 = json.load(open('data/students18.json'))
students19 = json.load(open('data/students19.json'))
students20 = json.load(open('data/students20.json'))
students21 = json.load(open('data/students21.json'))

In [15]:
# Function to check if two intervals intersect
def time_intersect(int1,int2):
    if int1[0] < int2[0] < int1[1] or int1[0] < int2[1] < int1[1]: # event 2 starts or ends during event 1
        return 1
    elif int2[0] < int1[0] < int2[1] or int2[0] < int1[1] < int2[1]: # event 1 starts or ends during event 2
        return 1
    else:
        return 0

# function to create a dictionnary with students as keys and list of events as values
def get_courses(students, events):
    print("Creating courses per student dictionary ...")
    courses = {}
    for event in tqdm(students):
        for student in students[event]:
            if student not in courses:
                courses[student] = [event]
            else:
                courses[student].append(event)

    for student in courses:
        courses[student] = sorted(courses[student], key=lambda x: events.loc[x]["Start"])
        
    return courses

# function to create a pandas dataframe with salles names as index and columns and the distance between them as values
def get_full_distances(distances, salles):
    print("Creating full distances dataframe ...")
    # create dataframe with all distances between all rooms 
    full_distances = np.zeros((len(salles), len(salles)))
    for i in tqdm(range(len(salles))):
        for j in range(len(salles)):
            if salles["Building"][i] == salles["Building"][j]:
                full_distances[i,j] = 0
            else:
                full_distances[i,j] = distances.loc[salles["Name"][i][0], salles["Name"][j][0]]
    full_distances = pd.DataFrame(full_distances, index=salles["Name"], columns=salles["Name"])

    return full_distances

# function to compute the number of students moving from one event to another 
def get_students_flow(courses, events):
    print("Creating students flow dataframe ...")
    flows = np.zeros((len(events), len(events)))
    flows = pd.DataFrame(flows, index=events.index, columns=events.index)
    for course in tqdm(courses):
        for i in range(len(courses[course])-1):
            flows.loc[courses[course][i], courses[course][i+1]] += 1
    return flows
    


In [16]:
courses18 = get_courses(students18, events18)
courses19 = get_courses(students19, events19)
courses20 = get_courses(students20, events20)
courses21 = get_courses(students21, events21)

Creating courses per student dictionary ...


100%|██████████| 490/490 [00:00<00:00, 13477.13it/s]


Creating courses per student dictionary ...


100%|██████████| 532/532 [00:00<00:00, 27040.68it/s]


Creating courses per student dictionary ...


100%|██████████| 510/510 [00:00<00:00, 42972.70it/s]


Creating courses per student dictionary ...


100%|██████████| 539/539 [00:00<00:00, 44574.51it/s]


In [17]:
full_distances = get_full_distances(distances, sallesDeCours)

Creating full distances dataframe ...


100%|██████████| 329/329 [00:02<00:00, 139.18it/s]


In [18]:
flows18 = get_students_flow(courses18, events18)
flows19 = get_students_flow(courses19, events19)
flows20 = get_students_flow(courses20, events20)
flows21 = get_students_flow(courses21, events21)

Creating students flow dataframe ...


100%|██████████| 17596/17596 [00:03<00:00, 5752.10it/s]


Creating students flow dataframe ...


100%|██████████| 17142/17142 [00:05<00:00, 3336.08it/s]


Creating students flow dataframe ...


100%|██████████| 17415/17415 [00:04<00:00, 4181.90it/s]


Creating students flow dataframe ...


100%|██████████| 17933/17933 [00:04<00:00, 4217.35it/s]


In [21]:
l_students18 = list(students18.keys())

l_events18 = list(events18.index)

l_rooms = list(sallesDeCours["Name"])

capacities = {}
for i in tqdm(range(len(sallesDeCours))):
    capacities[sallesDeCours["Name"][i]] = sallesDeCours["Capacity"][i]

nb_enrolled = {}
for i in tqdm(range(len(students18))):
    nb_enrolled[events18.index[i]] = len(students18[events18.index[i]])

flows_dict = {}
for i in tqdm(range(len(flows18))):
    flows_dict[flows18.index[i]] = {}
    for j in range(len(flows18)):
        flows_dict[flows18.index[i]][flows18.index[j]] = flows18.iloc[i,j]

distances_dict = {}
for i in tqdm(range(len(full_distances))):
    distances_dict[full_distances.index[i]] = {}
    for j in range(len(full_distances)):
        distances_dict[full_distances.index[i]][full_distances.index[j]] = full_distances.iloc[i,j]

100%|██████████| 329/329 [00:01<00:00, 168.18it/s]


In [32]:
# MIP Model
model18 = Model("room_allocation")

# Variables
# x_i,j = 1 if event i is in room j, 0 otherwise
x = model18.addVars(l_events18, l_rooms, vtype=GRB.BINARY, name="x")

# Constraints
# each event is in one room
model18.addConstrs((x.sum(i, '*') == 1 for i in l_events18), name="c1")

# each room has a capacity (multiplied by 1.2 to allow for 20% more students than capacity)
model18.addConstrs((x.prod(nb_enrolled, i, '*') <= 1.2*capacities[i] for i in l_rooms), name="c2")

# two events organized in the same room cannot intersect use the function time_intersect which returns 1 if two intervals intersect 
model18.addConstrs((x[i, j] + x[k, j] <= 2 - time_intersect([events18["Start"][i], events18["End"][i]], [events18["Start"][k], events18["End"][k]]) for i in l_events18 for j in l_rooms for k in l_events18 if i != k), name="c3")

# Objective
# minimize the distance between events for each student using full_distances and flows created above
model18.setObjective(x.prod(distances_dict, '*', '*') + x.prod(flows_dict, '*', '*'), GRB.MINIMIZE)

# Optimize
model18.optimize()


  2%|▏         | 12/490 [01:56<1:17:32,  9.73s/it]

KeyboardInterrupt: 

In [30]:
# print number of variables that are True
print('Number of variables that are True: %g' % sum([x[i,j].x for i in l_events18 for j in l_rooms]))

# print objective
print('Obj: %g' % model18.objVal)

Number of variables that are True: 490
Obj: 0
